In [1]:
# Standard libraries
import h5py
import numpy as np
import pandas as pd

# Append base directory
import os,sys,inspect
rootname = "mesoscopic-functional-connectivity"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

# User libraries
from codes.lib.signal_lib import downsample_int
from codes.lib.data_io.os_lib import getfiles_walk
from codes.lib.data_io.qt_wrapper import gui_fpath
from codes.lib.data_io.yaro.yaro_data_read import read_neuro_perf
from codes.lib.info_metrics.info_metrics_generic import parallel_metric_2d
from codes.lib.sweep_lib import DataSweep

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/mesoscopic-functional-connectivity


## 1. Load Neuronal Data

The neuronal data for a given experiment is given by the 3D matrix of dimensions `[N_Trial, N_Times, N_Channels]`

Notes:
* `DT = 0.05` : the sampling interval is always 50ms
* `N_Times = 201` : the total experiment time is always 10s. All experiments are aligned

In [2]:
NCore = 4
# Declare constants
params = {
    "raw_timestep" : 0.05, # 50ms, the standard measurement interval
    "bin_factor"   : 4,    # 200ms, factor by which the coarsen the time-resolution

    # The standard timesteps for different scenarios
    "range_CUE" : (1.0, 1.5),    # 1-1.5 seconds trial time
    "range_TEX" : (2.0, 3.5),    # 2-3.5 seconds trial time
    "range_LIK" : (3.5, 6.0),    # 3.5-6 seconds trial time
    "range_ALL" : (0.0, 10.0),   # 0-10  seconds

#     "samples_window" : "ALL",
    "trial_types" : ["iGO"],
#    "resample"    : {'method' : 'averaging', 'kind' : 'kernel'}  # None if raw data is prefered
    "resample" : True
}

# methods = ["BivariateMI", "MultivariateMI"]
methods = ["BivariateTE", "MultivariateTE"]

idtxlSettings = {
    'dim_order'       : 'rsp',
    'cmi_estimator'   : 'JidtGaussianCMI',
    'min_lag_sources' : 1,
    'max_lag_sources' : 5
}

In [3]:
# Get paths
root_path = gui_fpath("Path to root folder containing neuronal data", "./")
pathswalk = getfiles_walk(root_path, ['data.mat'])
datapaths = {os.path.basename(path) : path for path, file in pathswalk}
pd.DataFrame(datapaths, index=['dirname']).T

,dirname
m_01_2018_12_08_a,/media/alyosha/Data/TE_data/yarodata/TE_calcul...
m_01_2018_12_10_a,/media/alyosha/Data/TE_data/yarodata/TE_calcul...
m_03_2018_12_08_a,/media/alyosha/Data/TE_data/yarodata/TE_calcul...
m_03_2018_12_10_a,/media/alyosha/Data/TE_data/yarodata/TE_calcul...
m_04_2018_12_08_a,/media/alyosha/Data/TE_data/yarodata/TE_calcul...
m_04_2018_12_10_a,/media/alyosha/Data/TE_data/yarodata/TE_calcul...


In [4]:
out_path  = gui_fpath("Path where to save results", "./")

## Swipe Transfer Entropy via IDTxl

Notation of IDTxl
* `(processes, samples, replications)` equivalent to ours `(channels, times, trials)`
* Thus Yaro matrix `(nTrials, nTimes, nChannels)` is given by `'rsp'`, but
* For example `(nChannels, nTimes)` would be `'ps'`

In [5]:
for iFile, (folderName, folderPathName) in enumerate(datapaths.items()):
    #############################
    # Reading and downsampling
    #############################

    # Read LVM file from command line
    data, behaviour = read_neuro_perf(folderPathName, withPerformance=False, waitRetry=3, emulateBehaviour=True)

    # Get parameters
    nTrials, nTimes, nChannels = data.shape
    print("Loaded neuronal data with (nTrials, nTimes, nChannels)=", data.shape)

    #assert nTimes == 201, "The number of timesteps must be 201 for all data, got "+str(nTimes)


    # Timeline (x-axis)
    times = params["raw_timestep"] * np.linspace(0, nTimes, nTimes)

    # Downsample data
    if params["resample"]:
        params["timestep"] = params["raw_timestep"] * params["bin_factor"]
        print("Downsampling from", params["raw_timestep"], "ms to", params["timestep"], "ms")

        # Resample data
        timesDown, dataDown = downsample_int(times, data.transpose((1,0,2)), params["bin_factor"])

        # Replace old data with subsampled one
        times, data = timesDown, dataDown.transpose((1, 0, 2))
        nTrials, nTimes, nChannels = data.shape
        print("After downsampling data shape is (nTrials, nTimes, nChannels)=", data.shape)

    else:
        print("Skip resampling")
        params["timestep"] = params["raw_timestep"]
        

    for trialType in params['trial_types']:

        if trialType is None:
            dataEff = data
            fileNameSuffix = ""
        else:
            dataEff = data[behaviour[trialType] - 1]
            fileNameSuffix = "_" + trialType
            print("For trialType =", trialType, "the shape is (nTrials, nTimes, nChannels)=", dataEff.shape)

        #############################
        # Analysis
        #############################

        if dataEff.shape[0] < 50:
            print("Number of trials", dataEff.shape[0], "below threshold, skipping analysis")
        else:
            sweepSettings = {
                "window"    : idtxlSettings["max_lag_sources"] + 1, # 2
                "dim_order" : idtxlSettings["dim_order"]
            }
            dataSweep = DataSweep(data, sweepSettings)

            rez = parallel_metric_2d(dataSweep.iterator(), 'idtxl', methods, idtxlSettings, nCore=NCore)
            timesSweep = times[dataSweep.get_target_time_idxs()]
            nTimesEff = len(timesSweep)

            for methodName, methodRez in rez.items():
                te, lag, p = methodRez.transpose((1,2,3,0))

                #######################
                # Save results to file
                #######################
                savename = os.path.join(out_path, folderName + fileNameSuffix + '_' + methodName + '_swipe' + '.h5')
                print(savename)

                h5f = h5py.File(savename, "w")

                grp_rez = h5f.create_group("results")
                grp_rez['timestart']   = timesSweep[0]
                grp_rez['timestep']    = params["timestep"]
                grp_rez['TE_table']    = te
                grp_rez['delay_table'] = lag
                grp_rez['p_table']     = p

                h5f.close()

Reading Yaro data from /media/alyosha/Data/TE_data/yarodata/TE_calculation/m1/m_01_2018_12_08_a
Loaded neuronal data with (nTrials, nTimes, nChannels)= (74, 201, 12)
Downsampling from 0.05 ms to 0.2 ms
After downsampling data shape is (nTrials, nTimes, nChannels)= (74, 50, 12)
For trialType = iGO the shape is (nTrials, nTimes, nChannels)= (74, 50, 12)
Mem: 3359047680 - Start of subroutine
[2020.08.31 10:42:06] proc: 26836 mem: 3379351552 arg: 0 started
Redirecting STDOUT to log_26836.txt
[2020.08.31 10:42:06] proc: 26837 mem: 3382272000 arg: 8 started
Redirecting STDOUT to log_26837.txt
[2020.08.31 10:42:06] proc: 26838 mem: 3384463360 arg: 4 started
Redirecting STDOUT to log_26838.txt
[2020.08.31 10:42:06] proc: 26839 mem: 3389034496 arg: 0 started
Redirecting STDOUT to log_26839.txt
----Root process 26732 started task on 4 cores----
Resumed STDOUT
[2020.08.31 10:42:11] proc: 26839 mem: 4137033728 arg: 0 finished
[2020.08.31 10:42:11] proc: 26839 mem: 4137033728 arg: 1 started
Redirec

[2020.08.31 10:43:18] proc: 26838 mem: 3993325568 arg: 3 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 10:43:18] proc: 26836 mem: 3993452544 arg: 9 finished
[2020.08.31 10:43:18] proc: 26836 mem: 3993485312 arg: 10 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 10:43:23] proc: 26839 mem: 3993985024 arg: 11 finished
[2020.08.31 10:43:23] proc: 26839 mem: 3993985024 arg: 0 started
Redirecting STDOUT to log_26839.txt
Resumed STDOUT
[2020.08.31 10:43:23] proc: 26837 mem: 3993985024 arg: 6 finished
[2020.08.31 10:43:23] proc: 26837 mem: 3993985024 arg: 7 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 10:43:23] proc: 26838 mem: 3993989120 arg: 3 finished
[2020.08.31 10:43:23] proc: 26838 mem: 3993989120 arg: 4 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 10:43:24] proc: 26836 mem: 3993858048 arg: 10 finished
[2020.08.31 10:43:24] proc: 26836 mem: 3993858048 arg: 11 started
Redirecting STDOUT to

[2020.08.31 10:44:38] proc: 26836 mem: 4015357952 arg: 8 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 10:44:39] proc: 26839 mem: 4015357952 arg: 10 finished
[2020.08.31 10:44:39] proc: 26839 mem: 4015357952 arg: 11 started
Redirecting STDOUT to log_26839.txt
Resumed STDOUT
[2020.08.31 10:44:40] proc: 26838 mem: 4016451584 arg: 3 finished
[2020.08.31 10:44:40] proc: 26838 mem: 4016451584 arg: 4 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 10:44:44] proc: 26837 mem: 4012302336 arg: 5 finished
[2020.08.31 10:44:44] proc: 26837 mem: 4012302336 arg: 6 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 10:44:45] proc: 26839 mem: 4013187072 arg: 11 finished
[2020.08.31 10:44:45] proc: 26839 mem: 4013187072 arg: 0 started
Redirecting STDOUT to log_26839.txt
Resumed STDOUT
[2020.08.31 10:44:46] proc: 26838 mem: 4013436928 arg: 4 finished
[2020.08.31 10:44:46] proc: 26838 mem: 4013436928 arg: 5 started
Redirecting STDOUT to 

[2020.08.31 10:46:03] proc: 26838 mem: 3939446784 arg: 5 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 10:46:04] proc: 26837 mem: 3939573760 arg: 4 finished
[2020.08.31 10:46:04] proc: 26837 mem: 3939573760 arg: 5 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 10:46:05] proc: 26836 mem: 3939733504 arg: 4 finished
[2020.08.31 10:46:05] proc: 26836 mem: 3939733504 arg: 5 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 10:46:08] proc: 26839 mem: 3939414016 arg: 10 finished
[2020.08.31 10:46:08] proc: 26839 mem: 3939414016 arg: 11 started
Redirecting STDOUT to log_26839.txt
Resumed STDOUT
[2020.08.31 10:46:09] proc: 26837 mem: 3939831808 arg: 5 finished
[2020.08.31 10:46:09] proc: 26837 mem: 3939831808 arg: 6 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 10:46:09] proc: 26838 mem: 3939827712 arg: 5 finished
[2020.08.31 10:46:09] proc: 26838 mem: 3939827712 arg: 6 started
Redirecting STDOUT to l

[2020.08.31 10:47:21] proc: 26839 mem: 3928584192 arg: 10 started
Redirecting STDOUT to log_26839.txt
Resumed STDOUT
[2020.08.31 10:47:23] proc: 26838 mem: 3926528000 arg: 4 finished
[2020.08.31 10:47:23] proc: 26838 mem: 3926528000 arg: 5 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 10:47:24] proc: 26836 mem: 3926528000 arg: 1 finished
[2020.08.31 10:47:24] proc: 26836 mem: 3926528000 arg: 2 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 10:47:25] proc: 26837 mem: 3927035904 arg: 5 finished
[2020.08.31 10:47:25] proc: 26837 mem: 3927031808 arg: 6 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 10:47:27] proc: 26839 mem: 3927224320 arg: 10 finished
[2020.08.31 10:47:27] proc: 26839 mem: 3927224320 arg: 11 started
Redirecting STDOUT to log_26839.txt
Resumed STDOUT
[2020.08.31 10:47:29] proc: 26838 mem: 3927179264 arg: 5 finished
[2020.08.31 10:47:29] proc: 26838 mem: 3927179264 arg: 6 started
Redirecting STDOUT to 

...

[2020.08.31 14:47:08] proc: 26838 mem: 4139626496 arg: 11 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 14:47:10] proc: 26837 mem: 4139610112 arg: 10 finished
[2020.08.31 14:47:10] proc: 26837 mem: 4139610112 arg: 11 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 14:47:12] proc: 26836 mem: 4139737088 arg: 10 finished
[2020.08.31 14:47:12] proc: 26836 mem: 4139737088 arg: 11 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 14:47:13] proc: 26839 mem: 4139737088 arg: 8 finished
[2020.08.31 14:47:13] proc: 26839 mem: 4139737088 arg: 9 started
Redirecting STDOUT to log_26839.txt
Resumed STDOUT
[2020.08.31 14:47:15] proc: 26838 mem: 4139864064 arg: 11 finished
[2020.08.31 14:47:15] proc: 26838 mem: 4139859968 arg: 0 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 14:47:19] proc: 26839 mem: 4139982848 arg: 9 finished
[2020.08.31 14:47:19] proc: 26839 mem: 4139982848 arg: 10 started
Redirecting STDOUT

[2020.08.31 14:48:32] proc: 26836 mem: 4143161344 arg: 10 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 14:48:34] proc: 26839 mem: 4143341568 arg: 7 finished
[2020.08.31 14:48:34] proc: 26839 mem: 4143341568 arg: 8 started
Redirecting STDOUT to log_26839.txt
Resumed STDOUT
[2020.08.31 14:48:36] proc: 26838 mem: 4143501312 arg: 0 finished
[2020.08.31 14:48:36] proc: 26838 mem: 4143501312 arg: 1 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 14:48:38] proc: 26837 mem: 4148191232 arg: 7 finished
[2020.08.31 14:48:38] proc: 26837 mem: 4148191232 arg: 8 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 14:48:39] proc: 26836 mem: 4148191232 arg: 10 finished
[2020.08.31 14:48:39] proc: 26836 mem: 4148191232 arg: 11 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 14:48:40] proc: 26839 mem: 4148191232 arg: 8 finished
[2020.08.31 14:48:40] proc: 26839 mem: 4148191232 arg: 9 started
Redirecting STDOUT to 

[2020.08.31 14:50:05] proc: 26838 mem: 4154888192 arg: 1 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 14:50:06] proc: 26837 mem: 4154871808 arg: 7 finished
[2020.08.31 14:50:06] proc: 26837 mem: 4154871808 arg: 8 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 14:50:07] proc: 26836 mem: 4155019264 arg: 10 finished
[2020.08.31 14:50:07] proc: 26836 mem: 4155019264 arg: 11 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 14:50:12] proc: 26838 mem: 4155011072 arg: 1 finished
[2020.08.31 14:50:12] proc: 26838 mem: 4155011072 arg: 2 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 14:50:12] proc: 26837 mem: 4155015168 arg: 8 finished
[2020.08.31 14:50:12] proc: 26837 mem: 4155015168 arg: 9 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 14:50:13] proc: 26839 mem: 4155011072 arg: 3 finished
[2020.08.31 14:50:13] proc: 26839 mem: 4155011072 arg: 4 started
Redirecting STDOUT to l

[2020.08.31 14:51:33] proc: 26837 mem: 4165869568 arg: 10 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 14:51:34] proc: 26836 mem: 4165869568 arg: 9 finished
[2020.08.31 14:51:34] proc: 26836 mem: 4165869568 arg: 10 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 14:51:39] proc: 26839 mem: 4161966080 arg: 10 finished
[2020.08.31 14:51:39] proc: 26839 mem: 4161966080 arg: 11 started
Redirecting STDOUT to log_26839.txt
Resumed STDOUT
[2020.08.31 14:51:40] proc: 26837 mem: 4161875968 arg: 10 finished
[2020.08.31 14:51:40] proc: 26837 mem: 4161875968 arg: 11 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 14:51:41] proc: 26836 mem: 4161798144 arg: 10 finished
[2020.08.31 14:51:41] proc: 26836 mem: 4161798144 arg: 11 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 14:51:42] proc: 26838 mem: 4161892352 arg: 1 finished
[2020.08.31 14:51:42] proc: 26838 mem: 4161892352 arg: 2 started
Redirecting STDOU

Resumed STDOUT
[2020.08.31 14:52:59] proc: 26838 mem: 4045852672 arg: 0 finished
[2020.08.31 14:52:59] proc: 26838 mem: 4045852672 arg: 1 started
Redirecting STDOUT to log_26838.txt
Resumed STDOUT
[2020.08.31 14:53:00] proc: 26837 mem: 4045848576 arg: 9 finished
[2020.08.31 14:53:00] proc: 26837 mem: 4045848576 arg: 10 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 14:53:01] proc: 26836 mem: 4045717504 arg: 10 finished
[2020.08.31 14:53:01] proc: 26836 mem: 4045746176 arg: 11 started
Redirecting STDOUT to log_26836.txt
Resumed STDOUT
[2020.08.31 14:53:05] proc: 26837 mem: 4045975552 arg: 10 finished
[2020.08.31 14:53:05] proc: 26837 mem: 4045975552 arg: 11 started
Redirecting STDOUT to log_26837.txt
Resumed STDOUT
[2020.08.31 14:53:07] proc: 26836 mem: 4045979648 arg: 11 finished
Resumed STDOUT
[2020.08.31 14:53:10] proc: 26838 mem: 4046229504 arg: 1 finished
[2020.08.31 14:53:10] proc: 26838 mem: 4046229504 arg: 2 started
Redirecting STDOUT to log_26838.txt
Res